In [1]:
"""
Created on 30/12/2024

@author: federica montana

This file is used to calculate the indicators both at 250m resoltuion and city level for the 916 cities and 1 greater city (London) - 917 cities in total

"""
# Import packages
import pandas as pd
import os
os.environ['USE_PYGEOS'] = '0'
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd
import seaborn as sns
from utility import clusters, rescaling_dwellings
from libpysal import weights


## Urban design

### Optimal dwelling density

In [2]:
"""
Calculate the optimal dwelling density for each city at 250m resolution and city level
Columns of data:
- gid: Unique identifier for each row.
- pop_final: Population count.
- urau_code: Unique identifier for each urban area.
- urau_name: Name of the urban area.
- cluster: Cluster to which the urban area belongs.
- val_grid: Value of dwellings per hectare at 250m resolution.
- person_per_house: Number of persons per house.
"""
df = pd.read_csv('DATA/data/df_dwellings.csv')
df_popt = pd.read_csv('DATA\\data\\grid_pop.csv')
for cluster in clusters:
    # Read data
    df_odd = df[df.cluster==cluster]      

    # Grids      
    df1 = rescaling_dwellings(df_odd,'val_grid','rescaled_grid', target_val=6)
    df1.to_csv(f'DATA\\new_data\\{cluster}\\grid\\Urban Design_Optimal dwelling density.csv',index = False)

    # City
    df2 = df_odd.copy()
    df2['pop_city'] = df2.groupby('urau_code')['pop_final'].transform('sum')
    df2['dwelling_city'] = pd.to_numeric(df2['pop_city']/df2['person_per_house']) #number of housing 
    df_pop = df_popt[df_popt.cluster==cluster]
    df_pop['area'] = df_pop.groupby('urau_code')['urau_code'].transform('size') * 6.25 #area in ha
    df2 = pd.merge(df2, df_pop[['urau_code','gid','area']], on=['urau_code','gid'], how='left')
    df2['dwelling_city_ha'] = df2['dwelling_city'] / df2['area']
    df2.rename(columns={'dwelling_city_ha': 'val_city'}, inplace=True)
    #Take the city just once
    df2 = df2.drop_duplicates(subset=['urau_code'])

    #Divide the two dataset to apply differente rescale function
    df3 = rescaling_dwellings(df2,'val_city','rescaled_city', target_val=6)
    
    # Save data
    df3.to_csv(f'DATA\\new_data\\{cluster}\\city\\Urban Design_Optimal dwelling density.csv',index = False)

C:\Users\fmontana\AppData\Local\Temp\ipykernel_21796\3745889508.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pop['area'] = df_pop.groupby('urau_code')['urau_code'].transform('size') * 6.25 #area in ha
c:\Users\fmontana\Desktop\HUDI\Healthy-Urban-Design-Index\utility.py:195: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  noprov.loc[noprov['val_log'] < np.log(45), new_col] = np.interp(
c:\Users\fmontana\Desktop\HUDI\Healthy-Urban-Design-Index\utility.py:216: SettingWithCopyWarning: 
A value is tryin

### Compactness 


In [3]:
"""
Calculate Compactness for each city at city level using the gdftot which is the geodataframe having the population and the geometry of each grids
Columns of data:
- gid: Unique identifier for each row.
- pop_final: Population count.
- urau_code: Unique identifier for each urban area.
- urau_name: Name of the urban area.
- cluster: Cluster to which the urban area belongs.
"""
# --------------------------------------------------
# 1. Load your grid GeoDataFrame (250x250 m cells)

gdftot = gpd.read_file(f'DATA/data/gdf_pop.geojson').sort_values(by=['gid'], ascending=True)

# --------------------------------------------------
# Just ensure 'gdf' is in a projected CRS with meter units 
# (so areas make sense).
def classify_urban_clusters(gdf):
    # Thresholds based on 1,500/km² and 300/km²
    # for a 0.0625 km² cell (250m × 250m).
    hd_threshold = 93.75   # (1,500 inh./km² × 0.0625 km²)
    md_threshold = 18.75   # (300   inh./km² × 0.0625 km²)

    # Minimum total population for the entire cluster
    hd_min_pop = 50000  # Urban centre
    md_min_pop = 5000   # Urban cluster

    # --------------------------------------------------
    # 2. Identify Urban Centre Cells
    #    (High-Density: pop >= hd_threshold)
    # --------------------------------------------------
    gdf_hd = gdf[gdf['pop_final'] >= hd_threshold].copy()

    # Compute Rook contiguity only for these high-density cells
    if len(gdf_hd) > 0:
        w_rook = weights.contiguity.Rook.from_dataframe(gdf_hd)
        # Continue adjacency logic
        w_rook = weights.contiguity.Rook.from_dataframe(gdf_hd)
        rook_components = w_rook.component_labels
        gdf_hd['rook_comp'] = rook_components

        # Sum population in each rook-based cluster
        pop_by_rook_comp = gdf_hd.groupby('rook_comp')['pop_final'].sum()

        # Which clusters meet the total pop >= 50,000 ?
        urban_centre_components = pop_by_rook_comp[pop_by_rook_comp >= hd_min_pop].index
        # Mark those cells as "Urban Center"
        gdf_hd['classification'] = np.where(
            gdf_hd['rook_comp'].isin(urban_centre_components),
            'Urban Center',
            'Exclude'  # high-density cell not reaching 50k in total cluster
        )
    
    else:
        # No cells pass the threshold
        # Create empty placeholders or skip classification
        gdf_hd['rook_comp'] = []
        gdf_hd['classification'] = []


    # Merge classification back into main gdf
    gdf = gdf.merge(
        gdf_hd[['rook_comp', 'classification']], 
        left_index=True, 
        right_index=True, 
        how='left', 
        suffixes=('', '_hd')
    )

    # Explanation of the merge:
    #  - Some cells in gdf won’t be in gdf_hd if their pop < hd_threshold => classification will be NaN there.
    #  - Cells that belong to a cluster with total pop < 50k => classification = 'Exclude' (they didn’t make it to Urban Center).
    #  - Cells that made it => classification = 'Urban Center'.

    # --------------------------------------------------
    # 3. Identify Urban Cluster Cells (Moderate-Density)
    #    among cells NOT "Urban Center"
    # --------------------------------------------------
    # Filter out:
    #   - Cells already "Urban Center"
    #   - Cells below moderate threshold
    mask_md = (
        (gdf['classification'] != 'Urban Center') &
        (gdf['pop_final'] >= md_threshold)
    )
    gdf_md = gdf[mask_md].copy()

    if len(gdf_md) > 0:
        # Only do adjacency if there's something to analyze
        w_queen = weights.contiguity.Queen.from_dataframe(gdf_md)
        queen_components = w_queen.component_labels
        gdf_md['queen_comp'] = queen_components

        # Sum population in each queen-based cluster
        pop_by_queen_comp = gdf_md.groupby('queen_comp')['pop_final'].sum()

        # Which clusters meet total pop >= md_min_pop ?
        urban_cluster_components = pop_by_queen_comp[pop_by_queen_comp >= md_min_pop].index

        gdf_md['classification_md'] = np.where(
            gdf_md['queen_comp'].isin(urban_cluster_components),
            'Urban Cluster',
            'Exclude'
        )
    else: # cities without urban center
        # If empty, we can create empty columns or do nothing
        gdf_md['queen_comp'] = []
        gdf_md['classification_md'] = []

    # Merge back into the main gdf
    gdf = gdf.merge(
        gdf_md[['queen_comp', 'classification_md']],
        left_index=True,
        right_index=True,
        how='left',
        suffixes=('', '_md')
    )

    # --------------------------------------------------
    # 4. Final classification
    # --------------------------------------------------
    def final_classification(row):
        # Priority 1: Urban Center
        if row['classification'] == 'Urban Center':
            return 'Urban Center'
        
        # Priority 2: Urban Cluster
        elif row['classification_md'] == 'Urban Cluster':
            return 'Urban Cluster'
        
        # Otherwise, Rural
        else:
            return 'Rural'

    gdf['final_class'] = gdf.apply(final_classification, axis=1)
    
    # Compute the sprawl
    tot_pop = gdf.pop_final.sum()
    ucluster_pop = gdf[gdf.final_class == 'Urban Cluster'].pop_final.sum()/tot_pop
    ucenter_pop = gdf[gdf.final_class == 'Urban Center'].pop_final.sum()/tot_pop
    ruralcluster_pop = gdf[gdf.final_class == 'Rural'].pop_final.sum()/tot_pop

    gdf['sprawl'] = ((ruralcluster_pop + ucluster_pop - ucenter_pop) + 1 )*50
    # Compactness
    gdf['val_city'] = 100-gdf['sprawl'] 
    # Scale to have values [0-10]
    gdf['val_city'] = gdf['val_city'].round(0).astype(int)
    return gdf

# Suppose gdftot is your complete GeoDataFrame with many cities,
# having columns: 'urau_code', 'pop_final', geometry, etc.
from utility import clusters
for cluster in clusters:
    gdf2 = gdftot[gdftot.cluster == cluster]
    classified_all = gdf2.groupby('urau_code', group_keys=False).apply(
        classify_urban_clusters
    )
    classified_all = classified_all.drop_duplicates(subset='urau_code')
    classified_all = classified_all.copy()
    # Rescaled values
    classified_all['rescaled_city'] = classified_all['val_city']/10
    classified_all = classified_all[['gid','urau_code','urau_name', 'pop_final', 'val_city', 'rescaled_city','cluster']]
    classified_all.to_csv(f'DATA/new_data\\{cluster}\\city\\Urban Design_Compactness.csv', index=False) 

C:\Users\fmontana\AppData\Roaming\Python\Python39\site-packages\libpysal\weights\weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 42 disconnected components.
 There are 19 islands with ids: 10, 773, 992, 1225, 1273, 1415, 1983, 2429, 2481, 2635, 2664, 2811, 2877, 2963, 2990, 3315, 3330, 3339, 3477.
  warnings.warn(message)
C:\Users\fmontana\AppData\Roaming\Python\Python39\site-packages\libpysal\weights\weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 119 disconnected components.
 There are 68 islands with ids: 5, 20, 24, 25, 26, 27, 29, 32, 33, 35, 38, 40, 41, 42, 43, 45, 46, 47, 60, 61, 67, 68, 76, 77, 78, 81, 86, 87, 90, 91, 111, 123, 124, 125, 126, 128, 131, 140, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 155, 162, 167, 169, 170, 181, 186, 200, 201, 214, 234, 242, 247, 250, 262, 276, 285, 299.
  warnings.warn(message)
C:\Users\fmontana\AppData\Roaming\Python\Python39\site-packages\libpysal\weights

### Mid-rise development

In [4]:
"""
Calculate the Mid-rise development indicator for each city at 250m resolution and city level
Columns of df_lcz:
- gid: Unique identifier for each row.
- pop_final: Population count.
- urau_code: Unique identifier for each urban area.
- urau_name: Name of the urban area.
- cluster: Cluster to which the urban area belongs.
- val_grid: Numeric values containing low to mid-rise entries.
"""
df = pd.read_csv(f'DATA/data\df_lcz.csv')

for cluster in clusters:

    # Filter by cluster
    df_lcz = df[df.cluster == cluster]

    # Grid cell level
    # Scale to new range 0-10
    df1 = df_lcz.copy()
    MAX = df1['val_grid'].max()
    MIN = df1['val_grid'].min()
    df1['rescaled_grid'] = df1['val_grid'].apply(lambda x: (((x - MIN)/(MAX - MIN))*10))
    df1 = df1[['gid', 'urau_code','urau_name','pop_final', 'val_grid', 'rescaled_grid','cluster']]
    # Save data
    df1.to_csv(f'DATA/new_data\\{cluster}\\grid\\Urban Design_Mid-rise development.csv', index=False)

    #City level
    df2 = df_lcz.copy()
    # Calculate the population weighted average of the values for each city
    df2 = (df2.groupby('urau_code').apply(lambda x: (x['pop_final'] * x['val_grid']).sum()) / 
               df2.groupby('urau_code')['pop_final'].sum()).reset_index(name='val_city').merge(df2, on='urau_code', how='inner')
    # Take the city just once
    #df2['val_city'] =df2.groupby('urau_code')['val_grid'].transform('mean')
    df2.drop_duplicates(subset=['urau_code'], inplace=True)

    MAX = df2['val_city'].max()
    MIN = df2['val_city'].min()
    # Scale to new range 0-10
    df2['rescaled_city'] = df2['val_city'].apply(lambda x: (((x -MIN)/(MAX - MIN))*10))
    # Select columns
    df2 = df2[['gid','urau_code','urau_name', 'pop_final', 'val_city', 'rescaled_city','cluster']]
    # Save data
    df2.to_csv(f'DATA/new_data\\{cluster}\\city\\Urban Design_Mid-rise development.csv', index=False)

### Permeability

In [5]:
"""
Calculate the Permeability indicator for each city at 250m resolution and city level
Columns of df_imd:
- gid: Unique identifier for each row.
- pop_final: Population count.
- urau_code: Unique identifier for each urban area.
- urau_name: Name of the urban area.
- cluster: Cluster to which the urban area belongs.
- val_grid: Numeric values containing percentage of permeability data.
"""
df = pd.read_csv(f'DATA/data/df_imd.csv')
# Set the thresholds for rescaling
threshold = 25
# Set the new target value in the scale [0-10]
new_target = 6

for cluster in clusters:
    df_imd = df[df.cluster == cluster]
    # Grids
    # If values higher than threshold rescaled in [10-6], otherwise [5-0]
    df1 = df_imd.copy()
    MIN = df1['val_grid'].min()
    MAX = df1['val_grid'].max()
    # Apply interpolation for each range
    df['rescaled_grid'] = np.interp(
        df['val_grid'],
        [MIN, threshold, MAX],
        [0, 6, 10],
    )
    # Save data
    df.to_csv(f'DATA/new_data\\{cluster}\\grid\\Urban Design_Permeability.csv', index=False)

    # City
    df2 = df_imd.copy()
    df2['target'] = 0  # Initialize the column with default value 0
    df2.loc[df2['val_grid'] > 25, 'target'] = 1
    df2 = df2.copy()
    # Calculate the percentage of people meeting the grid permeability threshold
    df2['pop_target'] = df2['pop_final']*df2['target']
    df2['pop_sum_when_target_1'] = df2.groupby('urau_code')['pop_target'].transform('sum')
    df2['sum_pop'] = df2.groupby('urau_code')['pop_final'].transform('sum')
    df2['val_city'] = df2['pop_sum_when_target_1']/df2['sum_pop']*100
    # Rescale the values to the range [0-10]
    df2['rescaled_city'] = df2['val_city']/10 
    # Take the city just once
    df2 = df2.drop_duplicates('urau_name')
    df2 = df2[['gid','urau_code','urau_name', 'pop_final', 'val_city','rescaled_city','cluster']]

    df2.to_csv(f'DATA/new_data\\{cluster}\\city\\Urban Design_Permeability.csv', index=False)

# Sustainable Transportation

###  Opportunity to walk

In [6]:
"""
Calculate the Opportunity to walk indicator for each city at 250m resolution and city level
Columns of df_walk:
- gid: Unique identifier for each row.
- pop_final: Population count.
- urau_code: Unique identifier for each urban area.
- urau_name: Name of the urban area.
- cluster: Cluster to which the urban area belongs.
- val_grid: Numeric values containing % pedestrian infrastructure.
"""
df = pd.read_csv(f'DATA/data/df_walk.csv')
MIN_all = 0
MAX_all = 70
for cluster in clusters:

    df_ped = df[df.cluster == cluster]

    # Define the threshold for rescaling
    MAX = df_ped['val_grid'].max()
    MIN = df_ped['val_grid'].min()
    # Grid cell level
    
    # Apply rescaling directly without a function
    df_ped = df_ped.copy()
    df_ped['rescaled_grid'] = df_ped['val_grid'].apply(lambda x: 10 if x > MAX else (np.sqrt(x) / np.sqrt(MAX)) * 10)
                                                       #(((x -MIN)/(MAX - MIN))*10))
                                                       # lambda x: 10 if x > MAX else (np.sqrt(x) / np.sqrt(MAX)) * 10)
    df_ped = df_ped[['gid', 'urau_code','urau_name','pop_final', 'val_grid', 'rescaled_grid','cluster']]
    # Save data
    df_ped.to_csv(f'DATA/new_data\\{cluster}\\grid\\Sustainable Transportation_Opportunity to walk.csv', index=False)

    #City -- POPULATION WEIGHTED OPTIONS 
    df_ped2 = (df_ped.groupby('urau_code').apply(lambda x: (x['pop_final'] * x['val_grid']).sum()) / 
               df_ped.groupby('urau_code')['pop_final'].sum()).reset_index(name='val_city').merge(df_ped, 
                                                                                                          on='urau_code', how='inner')

    df_ped2 = df_ped2.drop_duplicates(subset=['urau_code'])
    df_ped2 = df_ped2.copy()
    df_ped2['rescaled_city'] = df_ped2['val_city'].apply(
        lambda x: 10 if x > MAX_all else (np.sqrt(x) / np.sqrt(MAX_all)) * 10)    
    #MIN = df_ped2['val_city'].min()
    #MAX = df_ped2['val_city'].max()
    #df_ped2['rescaled_city'] = df_ped2['val_city'].apply(lambda x: (np.sqrt(x) / np.sqrt(MAX)) * 10)
    df_ped2 = df_ped2[['gid','urau_code','urau_name', 'pop_final', 'val_city', 'rescaled_city','cluster']]
    df_ped2.to_csv(f'DATA/new_data\\{cluster}\\city\Sustainable Transportation_Opportunity to walk.csv', index=False) 


### Opportunity to cycle 

In [7]:
"""
Calculate the Opportunity to cycle indicator for each city at 250m resolution and city level
Columns of df_cycl:
- gid: Unique identifier for each row.
- pop_final: Population count.
- urau_code: Unique identifier for each urban area.
- urau_name: Name of the urban area.
- cluster: Cluster to which the urban area belongs.
- val_grid: Numeric values containing % cycling infrastructure.
"""
df = pd.read_csv(f'DATA/data/df_cycl.csv')
MIN_all = 0
MAX_all = 35
for cluster in clusters:

    df_cycl = df[df.cluster == cluster]

    # Define the threshold for rescaling
    MAX = df_cycl['val_grid'].max()
    MIN = df_cycl['val_grid'].min()

    # Grid cell level
    # Apply rescaling directly without a function
    df_cycl = df_cycl.copy()
    #df_cycl['rescaled_grid'] = df_cycl['val_grid'].apply(lambda x: (((x -MIN)/(MAX - MIN))*10))
    df_cycl['rescaled_grid'] = df_cycl['val_grid'].apply(
        lambda x: 10 if x > MAX else (np.sqrt(x) / np.sqrt(MAX)) * 10)
    df_cycl = df_cycl[['gid', 'urau_code','urau_name','pop_final', 'val_grid', 'rescaled_grid','cluster']]
    # Save data
    df_cycl.to_csv(f'DATA/new_data\\{cluster}\\grid\\Sustainable Transportation_Opportunity to cycle.csv', index=False)

    #City -- POPULATION WEIGHTED OPTIONS 
    df_cycl2 = (df_cycl.groupby('urau_code').apply(lambda x: (x['pop_final'] * x['val_grid']).sum()) / 
               df_cycl.groupby('urau_code')['pop_final'].sum()).reset_index(name='val_city').merge(df_cycl, 
                                                                                                          on='urau_code', how='inner')

    df_cycl2 = df_cycl2.drop_duplicates(subset=['urau_code'])
    df_cycl2 = df_cycl2.copy()
    df_cycl2['rescaled_city'] = df_cycl2['val_city'].apply(
        lambda x: 10 if x > MAX else (np.sqrt(x) / np.sqrt(MAX_all)) * 10)    
    #MIN = df_cycl2['val_city'].min()
    #MAX = df_cycl2['val_city'].max()
    #df_cycl2['rescaled_city'] = df_cycl2['val_city'].apply(lambda x: (np.sqrt(x) / np.sqrt(MAX)) * 10)  
    df_cycl2 = df_cycl2[['gid','urau_code','urau_name', 'pop_final', 'val_city', 'rescaled_city','cluster']]
    df_cycl2.to_csv(f'DATA/new_data\\{cluster}\\city\Sustainable Transportation_Opportunity to cycle.csv', index=False) 


### Public transport stops

In [8]:
"""
Calculate the Public transport stops indicator for each city at 250m resolution and city level
Columns of df_pubtrans:
- gid: Unique identifier for each row.
- pop_final: Population count.
- urau_code: Unique identifier for each urban area.
- urau_name: Name of the urban area.
- cluster: Cluster to which the urban area belongs.
- val_grid: Numeric values containing # of public transport stops.
"""
df = pd.read_csv(f'DATA/data/df_pubtrans.csv')
for cluster in clusters:
    df_bus = df[df.cluster == cluster]

    #Grid cell level
    df_bus = df_bus.copy()
    MIN = 0
    MAX = 20
    # Rescale such as 0 is 0 and 20 is 10
    df_bus['rescaled_grid'] = df_bus['val_grid'].apply(
        lambda x: 10 if x > MAX else (np.sqrt(x) / np.sqrt(MAX)) * 10
    )
    df_bus = df_bus[['gid', 'urau_code','urau_name','pop_final', 'val_grid', 'rescaled_grid','cluster']]
    df_bus.to_csv(f'DATA/new_data\\{cluster}\\grid\\Sustainable Transportation_Public transport stops.csv', index=False)

    #City  
    df_bus2 = df_bus.copy()
    df_bus2['present'] = 0  # Initialize the column with default value 0
    df_bus2.loc[df_bus2['val_grid'] > 0, 'present'] = 1  # Set value to 1 where val_grid > 0
    df_bus2['bus_tot_city'] = df_bus2.groupby('urau_code')['present'].transform('sum')
    df_bus2['grids'] = df_bus2.groupby('urau_code')['urau_name'].transform('size') 
    df_bus2 = df_bus2.drop_duplicates(subset=['urau_code'])
    df_bus2['bus_perc_city'] = df_bus2['bus_tot_city']/df_bus2['grids']*100
    df_bus2['rescaled_city'] = df_bus2['bus_perc_city']/10
    df_bus2 = df_bus2.drop_duplicates(subset=['urau_code'])
    df_bus2.rename(columns={'bus_perc_city': 'val_city'}, inplace=True)

    df_bus2 = df_bus2[['gid','urau_code','urau_name', 'pop_final', 'val_city','rescaled_city','cluster']]
    df_bus2.to_csv(f'DATA/new_data\\{cluster}\\city\Sustainable Transportation_Public transport stops.csv', index=False)


# Environmental Quality

### Air quality (PM2.5)

In [9]:
"""
Calculate the Air quality (PM2.5) indicator for each city at 250m resolution and city level
Columns of df_pm:
- gid: Unique identifier for each row.
- pop_final: Population count.
- urau_code: Unique identifier for each urban area.
- urau_name: Name of the urban area.
- cluster: Cluster to which the urban area belongs.
- val_grid: Numeric values containing PM2.5 values.
"""
df = pd.read_csv(f'DATA/data/df_pm.csv')
for cluster in clusters:
    
    df_ap = df[df.cluster == cluster]

    # Grid cell level
    MIN = 5 # this corespond to 10 in the rescaled grid
    MAX = df_ap['val_grid'].max() #corespond to 0 in the rescaled grid

    df_ap = df_ap.copy()

    df_ap['rescaled_grid'] = df_ap['val_grid'].apply(
        lambda x: 10 if x <= MIN else (0 if x >= MAX else 10 * (MAX - x) / (MAX - MIN))
    )
    df_ap.to_csv(f'DATA/new_data\\{cluster}\\grid\\Environmental Quality_Air quality (PM2.5).csv', index=False)

    #City level
    df_ap2 = df_ap.copy()

    # Aggregation
    df_ap2 = (df_ap.groupby('urau_code').apply(lambda x: (x['pop_final'] * x['val_grid']).sum()) / 
              df_ap.groupby('urau_code')['pop_final'].sum()).reset_index(name='val_city').merge(
                  df_ap, on='urau_code', how='inner')
      
    df_ap2 = df_ap2.drop_duplicates(subset=['urau_code'])

    MAX = df_ap2['val_city'].max()

    df_ap2 = df_ap2.copy()
    df_ap2['rescaled_city'] = df_ap2['val_city'].apply(
        lambda x: 10 if x <= MIN else (0 if x >= MAX else 10 * (MAX - x) / (MAX - MIN))
    )
    # Select columns
    df_ap2 = df_ap2[['gid','urau_code','urau_name', 'pop_final', 'val_city','rescaled_city','cluster']] 
    # Save 
    df_ap2.to_csv(f'DATA/new_data\\{cluster}\\city\\Environmental Quality_Air quality (PM2.5).csv', index=False)

### Air quality (NO2)

In [10]:
"""
Calculate the Air quality (NO2) for each city at 250m resolution and city level
Columns of df_no2:
- gid: Unique identifier for each row.
- pop_final: Population count.
- urau_code: Unique identifier for each urban area.
- urau_name: Name of the urban area.
- cluster: Cluster to which the urban area belongs.
- val_grid: Numeric values containing NO2 values.
"""
df = pd.read_csv(f'DATA/data/df_no2.csv')
for cluster in clusters:
    
    df_ap = df[df.cluster == cluster]

    # Grid cell level
    MIN = 10 # this corespond to 10 in the rescaled grid
    MAX = df_ap['val_grid'].max() #corespond to 0 in the rescaled grid

    df_ap = df_ap.copy()

    df_ap['rescaled_grid'] = df_ap['val_grid'].apply(
        lambda x: 10 if x <= MIN else (0 if x >= MAX else 10 * (MAX - x) / (MAX - MIN))
    )
    df_ap.to_csv(f'DATA/new_data\\{cluster}\\grid\\Environmental Quality_Air quality (NO2).csv', index=False)

    #City level
    df_ap2 = df_ap.copy()

    # Aggregation
    df_ap2 = (df_ap.groupby('urau_code').apply(lambda x: (x['pop_final'] * x['val_grid']).sum()) / 
              df_ap.groupby('urau_code')['pop_final'].sum()).reset_index(name='val_city').merge(
                  df_ap, on='urau_code', how='inner')
      
    df_ap2 = df_ap2.drop_duplicates(subset=['urau_code'])

    MAX = df_ap2['val_city'].max()

    df_ap2 = df_ap2.copy()
    df_ap2['rescaled_city'] = df_ap2['val_city'].apply(
        lambda x: 10 if x <= MIN else (0 if x >= MAX else 10 * (MAX - x) / (MAX - MIN))
    )
    # Select columns
    df_ap2 = df_ap2[['gid','urau_code','urau_name', 'pop_final', 'val_city','rescaled_city','cluster']] 
    # Save 
    df_ap2.to_csv(f'DATA/new_data\\{cluster}\\city\\Environmental Quality_Air quality (NO2).csv', index=False)

### Surrounding greenness (NDVI)

In [11]:
"""
Calculate the Surrounding greenness (NDVI) indicator for each city at 250m resolution and city level
Columns of df_ndvi:
- gid: Unique identifier for each row.
- pop_final: Population count.
- urau_code: Unique identifier for each urban area.
- urau_name: Name of the urban area.
- cluster: Cluster to which the urban area belongs.
- val_grid: Numeric values containing ndvi values.
"""
df = pd.read_csv(f'DATA/data/df_ndvi.csv')
df_target = pd.read_csv('DATA\\data\df_ndvi_target_tabla.csv')
MAX = 1
MIN = 0
for cluster in clusters:
    df_ndvi = df[df.cluster == cluster]
    
    # Grid
    # Grid
    df_ndvi = pd.merge(df_ndvi, df_target, on = 'urau_code') # Assuming target is the same for all rows in the group
    # Apply rescaling for each urau_code
    # Apply interpolation for each range
    df_ndvi['rescaled_grid'] = np.interp(
        df_ndvi['val_grid'],
        [MIN, threshold, MAX],
        [0, 6, 10],
    )
    df_ndvi2 = df_ndvi[['gid', 'urau_code','urau_name','pop_final', 'val_grid', 'rescaled_grid','cluster']]
    df_ndvi2.to_csv(f'DATA/new_data\\{cluster}\\grid\\Environmental Quality_Surrounding greenness.csv', index=False)

    # City
    df_ndvi2 = df_ndvi.copy()
    df_ndvi2['target'] = 0  # Initialize the column with default value 0
    # Set value to 1 where val_grid > target_ndvi
    df_ndvi2.loc[df_ndvi2['val_grid'] >= df_ndvi2['target_ndvi'], 'target'] = 1
    # Calculate the percentage of people meeting the grid NDVI threshold
    df_ndvi2['pop_target'] = df_ndvi2['pop_final']*df_ndvi2['target']
    df_ndvi2['pop_sum_when_target_1'] = df_ndvi2.groupby('urau_code')['pop_target'].transform('sum')
    df_ndvi2['sum_pop'] = df_ndvi2.groupby('urau_code')['pop_final'].transform('sum')
    df_ndvi2['val_city'] = df_ndvi2['pop_sum_when_target_1']/df_ndvi2['sum_pop']*100
    # Rescale the values to the range [0-10]
    df_ndvi2['rescaled_city'] = df_ndvi2['val_city']/10
    df_ndvi2 = df_ndvi2.drop_duplicates(subset=['urau_code'])
    # Select columns
    df_ndvi2 = df_ndvi2[['gid','urau_code','urau_name', 'pop_final', 'val_city', 'rescaled_city','cluster']]
    # Save 
    df_ndvi2.to_csv(f'DATA/new_data\\{cluster}\\city\\Environmental Quality_Surrounding greenness.csv', index=False)

### Lower Urban Heat Islands (UHI)

In [12]:
"""
Calculate the Lower Urban Heat Islands (UHI) indicator for each city at 250m resolution and city level
Columns of df_uhi:
- gid: Unique identifier for each row.
- pop_final: Population count.
- urau_code: Unique identifier for each urban area.
- urau_name: Name of the urban area.
- cluster: Cluster to which the urban area belongs.
- val_grid: Numeric values containing uhi values.
"""
df = pd.read_csv(f'DATA/data/df_uhi.csv')
for cluster in clusters:

    df_uhi = df[df.cluster == cluster]  

    # Grids
    MIN = df_uhi.val_grid.min()
    MAX = df_uhi.val_grid.max()
    
    df_uhi['rescaled_grid'] = df_uhi['val_grid'].apply(lambda x: 10-(((x - MIN)/( MAX - MIN))*10))
    df_uhi.to_csv(f'DATA/new_data\\{cluster}\\grid\\Environmental Quality_Lower urban heat islands.csv', index=False)

    #City
    df_uhi2 = (df_uhi.groupby('urau_code').apply(lambda x: (x['pop_final'] * x['val_grid']).sum()) / 
               df_uhi.groupby('urau_code')['pop_final'].sum()).reset_index(name='val_city')
    df_uhi2 = pd.merge(df_uhi2,df_uhi, on='urau_code', how='inner')
    df_uhi2 = df_uhi2.drop_duplicates(subset=['urau_code'])

    MAX = df_uhi2.val_city.max()
    MIN = df_uhi2.val_city.min()

    df_uhi2 = df_uhi2.copy()

    df_uhi2['rescaled_city'] = df_uhi2['val_city'].apply(
        lambda x: ((MAX-x)/(MAX - MIN))*10)
    # Select columns
    df_uhi2 = df_uhi2[['gid','urau_code','urau_name', 'pop_final', 'val_city', 'rescaled_city','cluster']]
    df_uhi2.to_csv(f'DATA/new_data\\{cluster}\\city\Environmental Quality_Lower urban heat islands.csv', index=False)


C:\Users\fmontana\AppData\Local\Temp\ipykernel_21796\1155148064.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_uhi['rescaled_grid'] = df_uhi['val_grid'].apply(lambda x: 10-(((x - MIN)/( MAX - MIN))*10))
C:\Users\fmontana\AppData\Local\Temp\ipykernel_21796\1155148064.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_uhi['rescaled_grid'] = df_uhi['val_grid'].apply(lambda x: 10-(((x - MIN)/( MAX - MIN))*10))
C:\Users\fmontana\AppData\Local\Temp\ipykernel_21796\1155148064.py:20: SettingWithCopyWa

## Green Accessibility

### Universal Access to Green Spaces

In [ ]:
"""
Calculate the Universal Access to Green Spaces indicator for each city at 250m resolution and city level
Columns of df_300m:
- gid: Unique identifier for each row.
- pop_final: Population count.
- urau_code: Unique identifier for each urban area.
- urau_name: Name of the urban area.
- cluster: Cluster to which the urban area belongs.
- val_grid: Numeric values containing log values of accessibility to green spaces.
- target_pop: Numeric values containing the target population.
"""
df = pd.read_csv(f'DATA\\data\\df_300m.csv')
# Read geodataframe
gdf = gpd.read_file(f'DATA/data/gdf_pop.geojson')
gdf1 = gdf.copy()
for cluster in clusters:

    df1 = df[df.cluster == cluster]

    # Rescale to a range from 0 to 10
    MIN = df1['val_grid'].min()
    MAX = df1['val_grid'].max()

    # Grid
    df1['rescaled_grid'] = 10 * (df1['val_grid'] - MIN) / (MAX - MIN)
    df1 = pd.merge(df1, gdf1[['gid','urau_code','pop_final','geometry']], on = ['gid','urau_code'], how = 'inner')
    # df3.loc[df3['urau_name'].str.contains(r'Bucure\?ti', regex=True), 'urau_name'] = 'Bucuresti'
    # df3.loc[df3['urau_name'].str.contains(r'Bucure?ti', regex=True), 'urau_name'] = 'Bucuresti'
    df2 = df1[['gid', 'urau_code','urau_name','pop_final', 'val_grid', 'rescaled_grid','cluster']]
    df2.to_csv(f'DATA/new_data\\{cluster}\\grid\\Green Spaces Accessibility_Universal access to green spaces.csv', index=False)

    #City
    df3 = df1.copy()
    df3['target_pop_sum'] = df3.groupby('urau_code')['target_pop'].transform('sum')
    df3['sum_pop'] = df3.groupby('urau_code')['pop_final'].transform('sum')
    df3['val_city'] = (df3['target_pop_sum']/df3['sum_pop'])*100
    df3 = df3.drop_duplicates(subset=['urau_code'])
    MIN = 0
    MAX = 100
    df3 = df3.copy()
    df3['rescaled_city'] = df3['val_city'].apply(lambda x: (((x - MIN)/(MAX - MIN))*10))
    df3 = df3[['gid','urau_code','urau_name', 'pop_final', 'val_city','rescaled_city','cluster']]


    df3.to_csv(f'DATA/new_data\\{cluster}\\city\Green Spaces Accessibility_Universal access to green spaces.csv', index=False)


C:\Users\fmontana\AppData\Local\Temp\ipykernel_27284\3617827850.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['rescaled_grid'] = 10 * (df1['val_grid'] - MIN) / (MAX - MIN)
C:\Users\fmontana\AppData\Local\Temp\ipykernel_27284\3617827850.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['rescaled_grid'] = 10 * (df1['val_grid'] - MIN) / (MAX - MIN)
C:\Users\fmontana\AppData\Local\Temp\ipykernel_27284\3617827850.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

### Access to Large Green Spaces

In [ ]:
"""
Calculate the Access to Large Green Spaces indicator for each city at 250m resolution and city level
Columns of df_2km:
- gid: Unique identifier for each row.
- pop_final: Population count.
- urau_code: Unique identifier for each urban area.
- urau_name: Name of the urban area.
- cluster: Cluster to which the urban area belongs.
- val_grid: Numeric values containing log values of accessibility to green spaces.
- target_pop: Numeric values containing the target population.
"""
df = pd.read_csv(f'DATA\\data\\df_2km.csv')
# Read geodataframe
# gdf1 = gpd.read_file(f'DATA/data/gdf_pop.geojson')
for cluster in clusters:

    df1 = df[df.cluster == cluster]

    # Rescale to a range from 0 to 10
    MIN = df1['val_grid'].min()
    MAX = df1['val_grid'].max()

    # Grid
    df1['rescaled_grid'] = 10 * (df1['val_grid'] - MIN) / (MAX - MIN)
    df1 = pd.merge(df1, gdf1[['gid','urau_code','pop_final','geometry']], on = ['gid','urau_code'], how = 'inner')
    # df3.loc[df3['urau_name'].str.contains(r'Bucure\?ti', regex=True), 'urau_name'] = 'Bucuresti'
    # df3.loc[df3['urau_name'].str.contains(r'Bucure?ti', regex=True), 'urau_name'] = 'Bucuresti'
    df2 = df1[['gid', 'urau_code','urau_name','pop_final', 'val_grid', 'rescaled_grid','cluster']]
    df2.to_csv(f'DATA/new_data\\{cluster}\\grid\\Green Spaces Accessibility_Access to large green spaces.csv', index=False)

    #City
    df3 = df1.copy()
    df3['target_pop_sum'] = df3.groupby('urau_code')['target_pop'].transform('sum')
    df3['sum_pop'] = df3.groupby('urau_code')['pop_final'].transform('sum')
    df3['val_city'] = (df3['target_pop_sum']/df3['sum_pop'])*100
    df3 = df3.drop_duplicates(subset=['urau_code'])
    MIN = 0
    MAX = 100
    df3 = df3.copy()
    df3['rescaled_city'] = df3['val_city'].apply(lambda x: (((x - MIN)/(MAX - MIN))*10))
    df3 = df3[['gid','urau_code','urau_name', 'pop_final', 'val_city','rescaled_city','cluster']]


    df3.to_csv(f'DATA/new_data\\{cluster}\\city\Green Spaces Accessibility_Access to large green spaces.csv', index=False)


C:\Users\fmontana\AppData\Local\Temp\ipykernel_27284\476383066.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['rescaled_grid'] = 10 * (df1['val_grid'] - MIN) / (MAX - MIN)
C:\Users\fmontana\AppData\Local\Temp\ipykernel_27284\476383066.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['rescaled_grid'] = 10 * (df1['val_grid'] - MIN) / (MAX - MIN)
C:\Users\fmontana\AppData\Local\Temp\ipykernel_27284\476383066.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from